<span style="font-size: 14pt">MIPT, Advanced ML, Autumn 2017</span>

<span style="font-size: 16pt"> HW #9: Recommendation System

<span style="color:blue; font-size: 12pt">Andrey Saitgalin </span>
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'">andrey.saitgalin@gmail.com</span>, 

<span style="color:blue; font-size: 12pt">Alexey Dral </span>,
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'">aadral@gmail.com</span>

<h1 align="center">Organization Info</h1> 

**Дополнительный материал для выполнения дз**:
<a href= "https://github.com/ml-mipt/ml-mipt-part2/tree/master/2017/lectures">Лекции по курсу</a>

**Оформление дз**: 
- Присылайте выполненное задание на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall <номер_группы> <фамилия>``, к примеру -- ``ML2017_Aut_fall 401 ivanov``
- Выполненное дз сохраните в файл ``<фамилия>_<группа>_task<номер>.ipnb``, к примеру -- ``ivanov_491_task9.ipnb``

**Вопросы**:
- Присылайте вопросы на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall Question <Содержание вопроса>``

** Дедлайн**:
<span style="color:red; font-size: 12pt">16.12.2017 23:59 </span>

--------
- **PS1**: Мы используем автоматические фильтры, и просто не найдем ваше дз, если вы не аккуратно его подпишите.
- **PS2**: Напоминаем, что дедлайны жесткие, письма пришедшие после автоматически удаляются =( чтобы соблазна не было.

<h1 align="center">Check Questions (0%)</h1> 
Вопросы отсутствуют

<h1 align="center"> Practical tasks (100%)</h1>


# Построение content-based рекомендательной системы
## Part 1
Для каждого фильма получите описание в текстовом виде с IMDB или (и) The movie database (у них есть открытое API). https://www.themoviedb.org/?language=ru. Вывести описание фильма под id 778. Также можно использовать рецензии.

In [11]:
import numpy as np
import pandas as pd
import tqdm
import pickle

In [12]:
import tmdbsimple
tmdbsimple.API_KEY = '6c1c603d2343091fbbdb92a259a88043'
movie = tmdbsimple.Movies(778)

In [13]:
movie.info()

{'adult': False,
 'backdrop_path': '/zF95knRgouUrBNbUh5oqRI5NIcE.jpg',
 'belongs_to_collection': None,
 'budget': 0,
 'genres': [{'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}],
 'homepage': '',
 'id': 778,
 'imdb_id': 'tt0046487',
 'original_language': 'fr',
 'original_title': 'Les vacances de Monsieur Hulot',
 'overview': 'Monsieur Hulot, Jacques Tati’s endearing clown, takes a holiday at a seaside resort, where his presence provokes one catastrophe after another. Tati’s masterpiece of gentle slapstick is a series of effortlessly well-choreographed sight gags involving dogs, boats, and firecrackers; it was the first entry in the Hulot series and the film that launched its maker to international stardom.',
 'popularity': 4.580258,
 'poster_path': '/r4F4tsU0Ajeh9ZYUkWOJSYmioj7.jpg',
 'production_companies': [{'id': 478, 'name': 'Cady'},
  {'id': 479, 'name': 'Discina'}],
 'production_countries': [{'iso_3166_1': 'FR', 'name': 'France'}],
 'release_date': '1953-02-26',
 'r

## Part 2 
Извлеките текстовые признаки у фильмов: tf idf, биграммы (может какие-то еще). Биграммы - статистика по биграммам, сколько какая биграмма встречается. Формулы для TF-IDF смотреть тут: https://en.wikipedia.org/wiki/Tf%E2%80%93idf 
    1. Используйте TF с double 0.5 normalization
    2. Используйте стандартный IDF: 
$$idf(t, D) = \frac{log N}{(1 + |d in D : t in d|)}$$

In [14]:
import http.client
import json
conn = http.client.HTTPSConnection("api.themoviedb.org")
payload = "{}"
conn.request("GET", "/3/movie/latest?language=en-US&api_key=6c1c603d2343091fbbdb92a259a88043", payload)
res = conn.getresponse()
data = json.loads(res.read().decode('utf-8'))

In [15]:
latest_id = 492957
data = pd.DataFrame(columns = data.keys())
links = pd.read_csv('links.csv')
for curr_film_id in tqdm.tqdm(links.tmdbId.values):
    try:
        curr_movie = tmdb.Movies(curr_film_id)
        data.loc[len(data)] = pd.Series(curr_movie.info()).values
    except:
        pass

100%|████████████████████████████████████████████████████████████████████████| 27278/27278 [00:00<00:00, 801700.10it/s]


In [19]:
data = pickle.load(open('fresh.pkl', 'rb'))
data = pd.DataFrame(data).T
data.index = np.arange(len(data))

In [25]:
from sklearn.feature_extraction.text import CountVectorizer as doit
overview = data.overview.values
co = doit()
counter = co.fit_transform(overview).astype(float)
tf = 0.5 * (counter.multiply(1. / counter.max(axis=1).todense()))
idf = np.log(tf.shape[0] / np.array(counter.sum(axis=0)))[0]
tf = tf.tocsr()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in true_divide
  """


In [26]:
bigram_vect = doit(ngram_range=(2,2))
bigram_counter = co.fit_transform(overview)
bigram_counter = bigram_counter.tocsr()

In [27]:
idf

array([  7.23474867,   4.8271108 ,  11.40913594, ...,  12.10228312,
        12.10228312,  12.10228312])

## Part 3 
Выведите подсчитанные признаки из пункта 2 для фильма с id 99114.

In [28]:
links = pd.read_csv('links.csv')
links[links.movieId == 99114]

,movieId,imdbId,tmdbId
20124,99114,1853728,68718.0


In [42]:
def count_tfidf(row_number):
    global tf
    global idf
    new_tf = np.array(tf[row_number].todense())[0] + 0.5
    return new_tf * idf

In [34]:
def count_bigram(row_number):
    global bigram_counter
    return np.array(bigram_counter[row_number].todense())[0]

In [35]:
def get_row_number_by_id(curr_id):
    return data[data.id == curr_id].index[0]

In [43]:
row_number = data[data.id == 68718.0].index[0]
count_tfidf(row_number)[:10]

array([ 3.61737433,  2.4135554 ,  5.70456797,  6.05114156,  6.05114156,
        6.05114156,  5.35799438,  6.05114156,  6.05114156,  6.05114156])

## Part 4
Для каждого пользователя найдите топ жанров, которые нравятся пользователю (можно делать как на семинаре top 5, top 3 и top 2) и порекомендуйте таким пользователя фильмы близкие по текстовому описанию (см. пункт 2). Для оценки результатов пункта 3 используйте метрику HitRate и метрики ранжирования (N)DCG. Формулу для подсчета HitRate можно взять на семинаре,  (N)DCG: https://en.wikipedia.org/wiki/Discounted_cumulative_gain



In [ ]:
# put your code here